In [2]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load intents data from JSON file
with open('Intents.json') as file:

    intents_data = json.load(file)

# Extract data from intents
patterns = []
responses = []
tags = []
for intent in intents_data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern.lower())  # Convert to lowercase for consistency
        responses.append(intent['responses'][0])  # Assuming only one response per pattern
        tags.append(intent['tag'])

# Tokenize input text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patterns)

vocab_size = len(tokenizer.word_index) + 1
max_len = max([len(seq) for seq in tokenizer.texts_to_sequences(patterns)])

# Convert text data into sequences
pattern_sequences = tokenizer.texts_to_sequences(patterns)

# Pad sequences to make them of equal length
pattern_pad = pad_sequences(pattern_sequences, maxlen=max_len, padding='post')

# Convert tags to categorical labels
tag_labels = np.array(tags)
label_indices = {tag: i for i, tag in enumerate(set(tags))}
tag_labels = [label_indices[tag] for tag in tag_labels]

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len),
    LSTM(64),
    Dense(len(label_indices), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(pattern_pad, np.array(tag_labels), epochs=100, verbose=2)

# Function to generate a response
# Function to generate a response
def generate_response(user_input):
    user_input_seq = tokenizer.texts_to_sequences([user_input.lower()])
    user_input_pad = pad_sequences(user_input_seq, maxlen=max_len, padding='post')
    predicted_probs = model.predict(user_input_pad)[0]
    predicted_tag = np.argmax(predicted_probs)
    predicted_tag_name = [tag for tag, index in label_indices.items() if index == predicted_tag][0]
    for intent in intents_data['intents']:
        if intent['tag'] == predicted_tag_name:
            return np.random.choice(intent['responses'])
            


Epoch 1/100
2/2 - 1s - loss: 2.7733 - accuracy: 0.0638 - 794ms/epoch - 397ms/step
Epoch 2/100
2/2 - 0s - loss: 2.7687 - accuracy: 0.0851 - 7ms/epoch - 3ms/step
Epoch 3/100
2/2 - 0s - loss: 2.7647 - accuracy: 0.1489 - 8ms/epoch - 4ms/step
Epoch 4/100
2/2 - 0s - loss: 2.7617 - accuracy: 0.2128 - 6ms/epoch - 3ms/step
Epoch 5/100
2/2 - 0s - loss: 2.7577 - accuracy: 0.3191 - 8ms/epoch - 4ms/step
Epoch 6/100
2/2 - 0s - loss: 2.7538 - accuracy: 0.3617 - 6ms/epoch - 3ms/step
Epoch 7/100
2/2 - 0s - loss: 2.7497 - accuracy: 0.3830 - 8ms/epoch - 4ms/step
Epoch 8/100
2/2 - 0s - loss: 2.7448 - accuracy: 0.3830 - 6ms/epoch - 3ms/step
Epoch 9/100
2/2 - 0s - loss: 2.7396 - accuracy: 0.3617 - 8ms/epoch - 4ms/step
Epoch 10/100
2/2 - 0s - loss: 2.7337 - accuracy: 0.3191 - 6ms/epoch - 3ms/step
Epoch 11/100
2/2 - 0s - loss: 2.7265 - accuracy: 0.3191 - 45ms/epoch - 22ms/step
Epoch 12/100
2/2 - 0s - loss: 2.7180 - accuracy: 0.3617 - 18ms/epoch - 9ms/step
Epoch 13/100
2/2 - 0s - loss: 2.7082 - accuracy: 0.383

In [ ]:
# Test the chatbot

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input)
    print("Bot:", response)